## **Imports**

In [19]:
%matplotlib inline
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import cv2
import torch   
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
# declate image size
imsize = 128

In [ ]:
# Mount to Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
! ls '/content/drive/My Drive/Mote_Manatee'

out2.npy  out3.npy  out.npy


In [5]:
# Read the npy file
training_data = np.load('/content/drive/My Drive/Mote_Manatee/out3.npy', allow_pickle=True)

In [6]:
training_data.shape

(1161, 3)

# **Dataset class loader**

In [7]:
class SiameseNetworkDataset(Dataset):
  pass

# **Neural Net Definition**

In [8]:
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 10),  # 64@96*96
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 64@48*48
            nn.Conv2d(64, 128, 7),
            nn.ReLU(),    # 128@42*42
            nn.MaxPool2d(2),   # 128@21*21
            nn.Conv2d(128, 128, 4),
            nn.ReLU(), # 128@18*18
            nn.MaxPool2d(2), # 128@9*9
            nn.Conv2d(128, 256, 4),
            nn.ReLU(),   # 256@6*6
        )
        self.liner = nn.Sequential(nn.Linear(9216, 4096), nn.Sigmoid())
        self.out = nn.Linear(4096, 1)

        # maybe add more linear layers to output 128

    def forward_one(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        output = self.liner(x)
        return output

    def forward(self, input1, input2):
        out1 = self.forward_one(input1)
        out2 = self.forward_one(input2)
        return out1, out2

In [9]:
net = Siamese()

# **Loss Function Definition**

In [10]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [107]:
"""
X2 = []
for labeled_pair in range(training_data.shape[0]):
  img1 = np.array(training_data[labeled_pair][1])
  # append
  X2.append(img1)
"""

## **Transform Data to PyTorch Tensors**

In [12]:
X1 = torch.Tensor([training_data[labeled_pair][0] for labeled_pair in range(training_data.shape[0] )]).view(-1, imsize, imsize)
X2 = torch.Tensor([training_data[labeled_pair][1] for labeled_pair in range(training_data.shape[0] )]).view(-1, imsize, imsize)
y = torch.Tensor([training_data[labeled_pair][2] for labeled_pair in range(training_data.shape[0] )])

print(X1.shape)
print(X2.shape)
print(y.shape)

torch.Size([1161, 128, 128])
torch.Size([1161, 128, 128])
torch.Size([1161, 2])


In [29]:
siamese_dataset[2].shape

torch.Size([1161, 2])

In [13]:
VAL_PCT = 0.1 # reserve 10% for validation
val_size = int(len(X1)*VAL_PCT)
print(val_size)

116


In [14]:
train_X1 = X1[:-val_size]
train_X2 = X2[:-val_size]
train_y = y[:-val_size]

test_X1 = X1[-val_size:]
test_X2 = X2[-val_size:]
test_y = y[-val_size:]

print(len(train_X1), len(train_X2))
print(len(test_X1), len(test_X2))

1045 1045
116 116


# **Train**

In [15]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = ContrastiveLoss()

In [47]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Running on the GPU")
else: 
  device = torch.device("cpu")
  print("Running on the CPU")

Running on the GPU


In [45]:
for epoch in range(EPOCHS):
    for X1,X2,y in data_loader:
        img0, img1 , label = data
        img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
        optimizer.zero_grad()
        output1,output2 = net(img0,img1)
        loss_function = criterion(output1,output2,label)
        loss_function.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())
show_plot(counter,loss_history)

RuntimeError: ignored